In [1]:
import os
import json

# Set paths
datasets_dir = '../../../datasets'
vqax_dir = os.path.join(datasets_dir, 'VQA-X')
train_dir = os.path.join(vqax_dir, 'vqaX_train.json')
test_dir = os.path.join(vqax_dir, 'vqaX_test.json')
val_dir = os.path.join(vqax_dir, 'vqaX_val.json')
with open(train_dir) as f:
    train_data = json.load(f)
with open(test_dir) as f:
    test_data = json.load(f)
with open(val_dir) as f:
    val_data = json.load(f)

In [4]:
print(json.dumps(test_data['262284001'],indent=2))

{
  "question": "What is this?",
  "answers": [
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 1
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 2
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 3
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 4
    },
    {
      "answer": "shower",
      "answer_confidence": "maybe",
      "answer_id": 5
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 6
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 7
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 8
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 9
    },
    {
      "answer": "shower",
      "answer_confidence": "yes",
      "answer_id": 10
    }
  ],
  "im

## Combine translations into one object

## Verifying the translations

In [12]:
import json
import os

# Đường dẫn đến thư mục chứa các file dữ liệu
datasets_dir = '../../../datasets/VQA-X'

# Danh sách các tập dữ liệu
datasets = ['train', 'val', 'test']

# Danh sách các nguồn dịch
translation_sources = ['vinai', 'gemini', 'ggtrans', 'gpt']

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def check_data_integrity(datasets_dir, datasets, translation_sources):
    for dataset in datasets:
        print(f"Checking {dataset} dataset...")
        
        # Load merged data
        merged_file = f'{datasets_dir}/vqaX_{dataset}_translated.json'
        merged_data = load_json(merged_file)
        
        error_items = {}
        
        for key, item in merged_data.items():
            item_errors = []
            
            # Check original fields
            if not item['question'] or not item['answers'] or not item['explanation']:
                item_errors.append("Missing original fields")
            
            # Check translated fields
            for source in translation_sources:
                question_key = f'question_vi_{source}'
                answer_key = f'answer_vi_{source}'
                explanation_key = f'explanation_vi_{source}'
                
                if question_key not in item or not item[question_key]:
                    item_errors.append(f"Missing or empty {question_key}")
                
                if answer_key not in item or not item[answer_key]:
                    item_errors.append(f"Missing or empty {answer_key}")
                
                if explanation_key not in item:
                    item_errors.append(f"Missing {explanation_key}")
                elif not item[explanation_key]:
                    item_errors.append(f"Empty {explanation_key}")
                elif len(item[explanation_key]) != len(item['explanation']):
                    item_errors.append(f"Mismatch in number of explanations for {explanation_key}")
                elif any(not exp for exp in item[explanation_key]):
                    item_errors.append(f"Empty explanation in {explanation_key}")
            
            if item_errors:
                error_items[key] = item_errors
        
        # Save error items
        error_file = f'{dataset}_errors.json'
        with open(error_file, 'w', encoding='utf-8') as f:
            json.dump(error_items, f, ensure_ascii=False, indent=2)
        
        print(f"Total items: {len(merged_data)}")
        print(f"Items with errors: {len(error_items)}")
        print("\n")

# Run the integrity check
check_data_integrity(datasets_dir, datasets, translation_sources)

Checking train dataset...
Total items: 29459
Items with errors: 0


Checking val dataset...
Total items: 1459
Items with errors: 0


Checking test dataset...
Total items: 1968
Items with errors: 0




In [11]:
import json
import os

# Đường dẫn đến thư mục chứa các file dữ liệu
datasets_dir = '../../../datasets/VQA-X'

# Danh sách các tập dữ liệu
datasets = ['train', 'val', 'test']

# Danh sách các nguồn dịch
translation_sources = ['vinai', 'gemini', 'ggtrans', 'gpt']

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def merge_translations(datasets_dir, datasets, translation_sources):
    for dataset in datasets:
        print(f"Processing {dataset} dataset...")
        
        # Load original data
        original_file = f'{datasets_dir}/vqaX_{dataset}.json'
        original_data = load_json(original_file)
        
        # Load translations
        translations = {}
        for source in translation_sources:
            translation_file = f'{datasets_dir}/vqaX_{dataset}_{source}.json'
            if os.path.exists(translation_file):
                translations[source] = load_json(translation_file)
            else:
                print(f"Warning: {translation_file} not found. Skipping this translation source.")
        
        # Merge translations
        merged_data = {}
        for key, item in original_data.items():
            merged_item = item.copy()
            for source in translation_sources:
                if source in translations and key in translations[source]:
                    translated_item = translations[source][key]
                    merged_item[f'question_vi_{source}'] = translated_item.get(f'question_vi_{source}')
                    merged_item[f'answer_vi_{source}'] = translated_item.get(f'answer_vi_{source}')
                    merged_item[f'explanation_vi_{source}'] = translated_item.get(f'explanation_vi_{source}')
            merged_data[key] = merged_item
        
        # Save merged data
        output_file = f'{datasets_dir}/vqaX_{dataset}_translated.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, ensure_ascii=False, indent=2)
        print(f"Merged data saved to {output_file}")
        
        # Print some statistics
        print(f"Total items in original data: {len(original_data)}")
        print(f"Total items in merged data: {len(merged_data)}")
        for source in translation_sources:
            if source in translations:
                print(f"Items with {source} translation: {sum(1 for item in merged_data.values() if f'question_vi_{source}' in item)}")
        print("\n")

# Run the merging process
merge_translations(datasets_dir, datasets, translation_sources)

Processing train dataset...
Merged data saved to ../../../datasets/VQA-X/vqaX_train_translated.json
Total items in original data: 29459
Total items in merged data: 29459
Items with vinai translation: 29459
Items with gemini translation: 29459
Items with ggtrans translation: 29459
Items with gpt translation: 29459


Processing val dataset...
Merged data saved to ../../../datasets/VQA-X/vqaX_val_translated.json
Total items in original data: 1459
Total items in merged data: 1459
Items with vinai translation: 1459
Items with gemini translation: 1459
Items with ggtrans translation: 1459
Items with gpt translation: 1459


Processing test dataset...
Merged data saved to ../../../datasets/VQA-X/vqaX_test_translated.json
Total items in original data: 1968
Total items in merged data: 1968
Items with vinai translation: 1968
Items with gemini translation: 1968
Items with ggtrans translation: 1968
Items with gpt translation: 1968




## Visualize argmax vs sampling

In [1]:
import os
import json
import pandas as pd
import random
random.seed(0)
from collections import Counter

def get_most_common_answer(answers):
    return Counter(answer['answer'] for answer in answers).most_common(1)[0][0]


def load_data(dataset_type, datasets_dir='../../../datasets/VQA-X/'):
    pp_dir = os.path.join(datasets_dir, 'final_data/post_processing')
    vote_dir = os.path.join(datasets_dir, 'final_data/argmax')

    with open(os.path.join(datasets_dir, f'vqaX_{dataset_type}_translated.json')) as f:
        data = json.load(f)

    argmax_data = pd.read_csv(os.path.join(vote_dir, f'{dataset_type}_argmax.csv'))
    sampling_data = pd.read_csv(os.path.join(pp_dir, f'vqaX_{dataset_type}_translated.csv'))

    return data, argmax_data, sampling_data

def print_sample_info(data, argmax_data, sampling_data, num_samples=100):
    translation_sources = ['vinai', 'gemini', 'ggtrans', 'gpt']

    indices = random.sample(list(data.keys()), num_samples)

    for idx in indices:
        print(f"ID: {idx}")
        print(f"Question: {data[idx]['question']}")
        print(f"Answers: {get_most_common_answer(data[idx]['answers'])}")
        for i, explanation in enumerate(data[idx]['explanation']):
            print(f"Explanation {i}: {explanation}")
            
            # Lấy dữ liệu từ argmax
            argmax_row = argmax_data[argmax_data['question_id'] == int(idx)]
            if not argmax_row.empty:
                print(f"Argmax: {argmax_row['explanation'].values[0].split('###')[i]}")
            
            # Lấy dữ liệu từ sampling
            sampling_row = sampling_data[sampling_data['question_id'] == int(idx)]
            if not sampling_row.empty:
                print(f"Sampling: {sampling_row['explanation'].values[0].split('###')[i]}")
            
            # In các translation từ các nguồn khác nhau
            for source in translation_sources:
                print(f"  {source}: {data[idx][f'explanation_vi_{source}'][i]}")
            print()
        print("="*100)

def process_dataset(dataset_type, num_samples=100):
    data, argmax_data, sampling_data = load_data(dataset_type)
    print_sample_info(data, argmax_data, sampling_data, num_samples)

process_dataset('train', num_samples=100)


ID: 122964003
Question: What kind of room is this?
Answers: kitchen
Explanation 0: you can use the nice sink to wash vegetables
Argmax: bạn có thể sử dụng bồn rửa đẹp để rửa rau
Sampling: bạn có thể sử dụng bồn rửa đẹp để rửa rau củ
  vinai: bạn có thể sử dụng bồn rửa đẹp để rửa rau
  gemini: bạn có thể sử dụng bồn rửa tiện lợi để rửa rau
  ggtrans: bạn có thể sử dụng bồn rửa đẹp để rửa rau
  gpt: bạn có thể sử dụng bồn rửa đẹp để rửa rau củ

ID: 184999000
Question: Is this mall crowded?
Answers: no
Explanation 0: there are no people walking around
Argmax: không có người đi lại xung quanh
Sampling: không có người đi bộ xung quanh
  vinai: không có người đi bộ xung quanh
  gemini: không có ai đi bộ xung quanh
  ggtrans: không có người đi lại xung quanh
  gpt: không có ai đi lại

ID: 192954004
Question: What room is this?
Answers: living room
Explanation 0: the fireplace is warming the couch and chair
Argmax: lò sưởi đang sưởi ấm ghế sofa và ghế
Sampling: lò sưởi đang sưởi ấm ghế sofa và

In [3]:
# add img_id column to argmax and sampling data
import os
import json
import pandas as pd
datasets_dir = "../../../datasets"
vqax_dir = os.path.join(datasets_dir, "VQA-X")
sampling_dir = os.path.join(vqax_dir, "final_data/sampling")
datasets = ["train", "val", "test"]

for dataset in datasets:
    print(f"Processing {dataset} dataset...")
    sampling_file = os.path.join(sampling_dir, f"{dataset}_sampling.csv")
    orginal_file = os.path.join(vqax_dir, f"vqaX_{dataset}.json")
    sampling_data = pd.read_csv(sampling_file)
    with open(orginal_file) as f:
        original_data = json.load(f)
    sampling_data["img_id"] = sampling_data["question_id"].apply(lambda x: original_data[str(x)]["image_name"].split(".")[0])
    # sampling_data.to_csv(os.path.join(vqax_dir, f"{dataset}_sampling.csv"), index=False, encoding="utf-8", columns=["question_id", "img_id", "question", "answer", "explanation"])
    sampling_data.to_csv(sampling_file, index=False, encoding="utf-8", columns=["question_id", "img_id", "question", "answer", "explanation"])

    print(f"Saved {sampling_file}")
    
    

Processing train dataset...
Saved ../../../datasets/VQA-X/final_data/sampling/train_sampling.csv
Processing val dataset...
Saved ../../../datasets/VQA-X/final_data/sampling/val_sampling.csv
Processing test dataset...
Saved ../../../datasets/VQA-X/final_data/sampling/test_sampling.csv


## Check the score of the translations

In [1]:
import os
import json

# Đường dẫn tới thư mục datasets và evaluation
datasets_dir = '../../../datasets/VQA-X'
evaluation_dir = os.path.join(datasets_dir, 'evaluation')
llm_models = ['llama', 'qwen', 'phi','gemma','gpt']
evaluation_files = ['val', 'test', 'train']

# Hàm kiểm tra các trường score
def check_scores_length(data):
    errors = []
    for entry in data:
        if len(entry['question_scores']) != len(entry['question']):
            errors.append(f"Mismatch in question_scores: {entry['id_image']}")
        if len(entry['answer_scores']) != len(entry['answer']):
            errors.append(f"Mismatch in answer_scores: {entry['id_image']}")
        for i, expl_scores in enumerate(entry['explanation_scores']):
            try:
                if len(expl_scores) != len(entry['explanation'][i]):
                    errors.append(f"Mismatch in explanation_scores[{i}]: {entry['id_image']}")
            except:
                errors.append(f"Error in explanation_scores[{i}]: {entry['id_image']}")

    return errors

# Kiểm tra các file JSON của từng LLM
for model in llm_models:
    for eval_file in evaluation_files:
        file_path = os.path.join(evaluation_dir, f"{eval_file}_{model}_evaluation.json")
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)            
                errors = check_scores_length(data)
                if errors:
                    print(f"Errors in {file_path}:")
                    for error in errors:
                        print(f"  - {error}")
                else:
                    print(f"No errors found in {eval_file}_{model}_evaluation.json")
        else:
            print(f"File {file_path} does not exist.")

No errors found in val_llama_evaluation.json
No errors found in test_llama_evaluation.json
No errors found in train_llama_evaluation.json
No errors found in val_qwen_evaluation.json
No errors found in test_qwen_evaluation.json
No errors found in train_qwen_evaluation.json
No errors found in val_phi_evaluation.json
No errors found in test_phi_evaluation.json
No errors found in train_phi_evaluation.json
No errors found in val_gemma_evaluation.json
No errors found in test_gemma_evaluation.json
No errors found in train_gemma_evaluation.json
No errors found in val_gpt_evaluation.json
No errors found in test_gpt_evaluation.json
No errors found in train_gpt_evaluation.json


## Merge the scores

In [1]:
import json
import os

datasets_dir = '../../../datasets/VQA-X'
evaluation_dir = os.path.join(datasets_dir, 'evaluation')
llm_models = ['llama', 'qwen', 'phi', 'gemma', 'gpt']
evaluation_files = ['train', 'val', 'test']

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def merge_evaluation_data(**json_files):
    merged_data = {}
    
    for model_name, file_path in json_files.items():
        data = load_json(file_path)
        
        for entry in data:
            question_id = entry["id_image"] # need to change to question_id
            if question_id not in merged_data:
                merged_data[question_id] = {
                    "question_id": question_id,
                    "question": entry["question"],
                    "question_scores": {},
                    "answer": entry["answer"],
                    "answer_scores": {},
                    "explanation": entry["explanation"],
                    "explanation_scores": {model_name: entry["explanation_scores"]}
                }
            
            # Add scores with the model name as the key
            merged_data[question_id]["question_scores"][model_name] = entry["question_scores"]
            merged_data[question_id]["answer_scores"][model_name] = entry["answer_scores"]
        
            merged_data[question_id]["explanation_scores"][model_name] = entry["explanation_scores"]

    return list(merged_data.values())

# Process each dataset (train, val, test)
for evaluation_file in evaluation_files:
    # Define paths to the JSON files for each model and the current evaluation file
    json_files = {
        model: os.path.join(evaluation_dir, f"{evaluation_file}_{model}_evaluation.json")
        for model in llm_models
    }
    
    # Merge the data for the current evaluation file
    merged_data = merge_evaluation_data(**json_files)
    
    # Save the merged data
    output_path = os.path.join(evaluation_dir, f'vqaX_{evaluation_file}_evaluation.json')
    with open(output_path, 'w', encoding='utf-8') as outfile:
        json.dump(merged_data, outfile, indent=2, ensure_ascii=False)
    
    print(f"Merged data for {evaluation_file} saved to {output_path}")


Merged data for train saved to ../../../datasets/VQA-X/evaluation/vqaX_train_evaluation.json
Merged data for val saved to ../../../datasets/VQA-X/evaluation/vqaX_val_evaluation.json
Merged data for test saved to ../../../datasets/VQA-X/evaluation/vqaX_test_evaluation.json


## Test gpt

In [13]:
import openai
import json
from dotenv import load_dotenv
import os
load_dotenv()
# Đặt API key của bạn
openai.api_key = os.getenv("OPENAI_API_KEY")
prompt = """
You will be given an English question, answer, and explanations for context. Then, you will evaluate Vietnamese translations of the question, answer, and explanation(s). Evaluate each translation based on accuracy, fluency, and cultural appropriateness, considering the full context provided. Assign a score between 0 and 100 for each translation.

**Return the scores in the following JSON format and no additional text or explanations:**

{{
    "question_scores": [score_for_translation_1, score_for_translation_2, ...],
    "answer_scores": [score_for_translation_1, score_for_translation_2, ...],
    "explanation_scores": [
        [score for translations of explanation 1],
        [score for translations of explanation 2], (if multiple explanations are provided)
        ...
    ] (length of this list should match the number of explanations provided)
}}

Now, please evaluate the following:
"""
content_user = """
English Question: How many legs does a cat have?
Translation 1: Mèo có bao nhiêu chân?
Translation 2: Mèo có mấy chân?
Translation 3: Mèo có mấy cái chân?

English Answer: A cat has four legs.
Translation 1: Mèo có bốn chân.
Translation 2: Mèo có 4 chân.
Translation 3: Mèo có bốn cái chân.

Explanation 1: Cats are quadrupedal animals.
Translation 1: Mèo là động vật có bốn chân.
Translation 2: Mèo là loài động vật có bốn chân.
Translation 3: Mèo là loài động vật có bốn cái chân.
"""


messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": content_user}
]

# Gọi API
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    max_tokens=100,
    temperature=0.1,
    response_format={"type": "json_object"}
)


In [14]:
print(response.choices[0].message.content)
json_object = json.loads(response.choices[0].message.content)
json_object['explain_score']

{
    "question_score": [90, 95, 85],
    "answer_score": [95, 95, 85],
    "explain_score": [
        [90, 95, 85]
    ]
}


[[90, 95, 85]]

## Evaluate predictions

In [10]:
import json
import os
import pandas as pd
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from fairseq.data.encoders.gpt2_bpe import GPT2BPE, GPT2BPEConfig

# Tạo đối tượng BPE với đường dẫn đến các file vocab và encoder.json
bpe_cfg = {
    'gpt2_encoder_json': '/home/VLAI/binhdt/OFA-X/OFA/utils/BPE/encoder.json',
    'gpt2_vocab_bpe': '/home/VLAI/binhdt/OFA-X/OFA/utils/BPE/vocab.bpe'
}
bpe = GPT2BPE(GPT2BPEConfig(bpe_cfg))

# Hàm giải mã BPE
def decode_bpe(encoded_tokens, bpe):
    decoded_text = bpe.decode(' '.join(map(str, encoded_tokens)))
    return decoded_text

def encode_bpe(text, bpe):
    encoded_tokens = bpe.encode(text)
    return encoded_tokens

# Đọc file JSON
test_predict_file = '/home/VLAI/binhdt/OFA-X/results/bz_32_0.4161/test_predict.json'
with open(test_predict_file, 'r', encoding='utf-8') as f:
    test_predict = json.load(f)

predict_df = pd.DataFrame(test_predict)

# Áp dụng giải mã cho cột target_explanation và explanation (chỉ nếu là danh sách BPE)
predict_df['target_explanation'] = predict_df['target_explanation'].apply(lambda x: decode_bpe(x, bpe))
# predict_df['explanation'] = predict_df['explanation'].apply(lambda x: encode_bpe(x, bpe))
# # chuyển list sang string
# predict_df['target_explanation'] = predict_df['target_explanation'].apply(lambda x: ' '.join(map(str, x)))
predict_df.head()

,id,answer,explanation,target_explanation
0,262284001,phòng tắm,vì có vòi hoa sen và vòi hoa sen,desinter shaped�cert desinter Hamiltonion fre...
1,262284001,phòng tắm,vì có vòi hoa sen và vòi hoa sen,desinter shaped�cert desinter Hamiltonrough f...
2,262284001,phòng tắm,vì có vòi hoa sen và vòi hoa sen,desinter shaped�cert desinter Hamiltonrough f...
3,393372017,không,vì cô ấy đang mỉm cười và cô ấy đang đứng trên...,agreeunny visual answ desinter Hamilton butar...
4,393372017,không,vì cô ấy đang mỉm cười và cô ấy đang đứng trên...,agreeunny visual answ desinter Hamiltonroughj...


In [11]:
# Chuẩn bị dữ liệu tham chiếu (references) và dự đoán (hypotheses)
gts = {idx: [ref] for idx, ref in enumerate(predict_df['target_explanation'])}
res = {idx: [hyp] for idx, hyp in enumerate(predict_df['explanation'])}

# Khởi tạo các đối tượng tính điểm
bleu_scorer = Bleu(4)  # Bleu-1 đến Bleu-4
meteor_scorer = Meteor()
rouge_scorer = Rouge()
cider_scorer = Cider()

# Tính điểm Bleu
bleu_score, bleu_scores = bleu_scorer.compute_score(gts, res)
print(f"Bleu Scores: {bleu_score}")

# Tính điểm METEOR
meteor_score, meteor_scores = meteor_scorer.compute_score(gts, res)
print(f"METEOR Score: {meteor_score}")

# Tính điểm ROUGE_L
rouge_score, rouge_scores = rouge_scorer.compute_score(gts, res)
print(f"ROUGE_L Score: {rouge_score}")

# Tính điểm CIDEr
cider_score, cider_scores = cider_scorer.compute_score(gts, res)
print(f"CIDEr Score: {cider_score}")

{'testlen': 56313, 'reflen': 121129, 'guess': [56313, 50412, 44511, 38610], 'correct': [0, 0, 0, 0]}
ratio: 0.46490105589907893
Bleu Scores: [5.617206829634975e-21, 5.936873340529306e-21, 6.303654937497847e-21, 6.730566868823769e-21]
METEOR Score: 0.0002329689443605222
ROUGE_L Score: 5.049042473869292e-05
CIDEr Score: 0.0


In [12]:
import pandas as pd
test_file = '/home/VLAI/binhdt/OFA-X/data/vqax/vqaX_test_translated.tsv'
test_df = pd.read_csv(test_file, sep='\t', header=None, names=['question_id', 'img_id', 'question', 'conf|!+answer', 'explanation', 'empty', 'base64_encoded_image'])

In [13]:
import json
import os
import pandas as pd
test_predict_file = '/home/VLAI/binhdt/OFA-X/results/bz_32_0.4161/test_predict.json'
with open(test_predict_file, 'r', encoding='utf-8') as f:
    test_predict = json.load(f)
    
predict_df = pd.DataFrame(test_predict)
predict_df.drop(columns=['target_explanation'], inplace=True)
grouped_target = test_df[['question_id', 'explanation']].groupby('question_id')['explanation'].apply(list).reset_index()

# Chuyển 'question_id' thành 'id' để có thể merge với 'predict_df'
grouped_target = grouped_target.rename(columns={'question_id': 'id'})

# Convert 'id' columns to the same data type
grouped_target['id'] = grouped_target['id'].astype(str)
predict_df['id'] = predict_df['id'].astype(str)

# Merge các câu giải thích từ test.tsv vào prediction df
merged_df = pd.merge(predict_df, grouped_target, how='left', on='id')

# "Explode" cột 'target_explanation' để mỗi hàng chỉ chứa một giải thích
merged_df = merged_df.explode('explanation_y')

# Đặt lại tên cột để cột 'explanation_y' trở thành 'target_explanation'
merged_df = merged_df.rename(columns={'explanation_y': 'target_explanation'})
merged_df = merged_df.rename(columns={'explanation_x': 'explanation'})
merged_df['explanation'] = merged_df['explanation'].apply(lambda x: x.lstrip("vì "))
# target_answer
merged_df['target_answer'] = test_df['conf|!+answer'].apply(lambda x: x.split('|!+')[1])
# drop duplicate
merged_df = merged_df.drop_duplicates(subset=['id', 'answer', 'explanation', 'target_explanation'])
merged_df.head()

,id,answer,explanation,target_explanation,target_answer
0,262284001,phòng tắm,có vòi hoa sen và vòi hoa sen,nó có vòi hoa sen treo bên trong,vòi sen
0,262284001,phòng tắm,có vòi hoa sen và vòi hoa sen,có vòi hoa sen,vòi sen
0,262284001,phòng tắm,có vòi hoa sen và vòi hoa sen,có vòi nước và bồn tắm,vòi sen
3,393372017,không,cô ấy đang mỉm cười và cô ấy đang đứng trên đầu,mắt cô ấy mở và hướng về phía trước,Đúng
3,393372017,không,cô ấy đang mỉm cười và cô ấy đang đứng trên đầu,cô ấy đang hướng về phía máy ảnh và tạo dáng t...,Đúng


In [14]:
import regex as re
def clean(st):
    # remove everything in between parentheses
    return re.sub(r" \([^)]*\)", "", st)

# clean merge df
merged_df['answer'] = merged_df['answer'].apply(lambda x: clean(x))
merged_df['explanation'] = merged_df['explanation'].apply(lambda x: clean(x))
merged_df['target_explanation'] = merged_df['target_explanation'].apply(lambda x: clean(x))
merged_df['target_answer'] = merged_df['target_answer'].apply(lambda x: clean(x))
merged_df.head()

,id,answer,explanation,target_explanation,target_answer
0,262284001,phòng tắm,có vòi hoa sen và vòi hoa sen,nó có vòi hoa sen treo bên trong,vòi sen
0,262284001,phòng tắm,có vòi hoa sen và vòi hoa sen,có vòi hoa sen,vòi sen
0,262284001,phòng tắm,có vòi hoa sen và vòi hoa sen,có vòi nước và bồn tắm,vòi sen
3,393372017,không,cô ấy đang mỉm cười và cô ấy đang đứng trên đầu,mắt cô ấy mở và hướng về phía trước,Đúng
3,393372017,không,cô ấy đang mỉm cười và cô ấy đang đứng trên đầu,cô ấy đang hướng về phía máy ảnh và tạo dáng t...,Đúng


In [15]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

# Chuẩn bị dữ liệu tham chiếu (references) và dự đoán (hypotheses)
gts = {idx: [ref] for idx, ref in enumerate(merged_df['target_explanation'].tolist())}
res = {idx: [hyp] for idx, hyp in enumerate(merged_df['explanation'].tolist())}

# Khởi tạo các đối tượng tính điểm
bleu_scorer = Bleu(4)  # Bleu-1 đến Bleu-4
meteor_scorer = Meteor()
rouge_scorer = Rouge()
cider_scorer = Cider()

# Tính điểm Bleu
bleu_score, bleu_scores = bleu_scorer.compute_score(gts, res)
for i, score in enumerate(bleu_score):
    print(f"Bleu-{i+1} Score: {score:.4f}")

# Tính điểm METEOR
meteor_score, meteor_scores = meteor_scorer.compute_score(gts, res)
print(f"METEOR Score: {meteor_score:.4f}")

# Tính điểm ROUGE_L
rouge_score, rouge_scores = rouge_scorer.compute_score(gts, res)
print(f"ROUGE_L Score: {rouge_score:.4f}")

# Tính điểm CIDEr
cider_score, cider_scores = cider_scorer.compute_score(gts, res)
print(f"CIDEr Score: {cider_score:.4f}")

# Tính accuracy
accuracy = (merged_df['target_answer'] == merged_df['answer']).mean()
print(f"Accuracy: {accuracy:.4f}")

{'testlen': 50178, 'reflen': 60729, 'guess': [50178, 44305, 38432, 32559], 'correct': [17095, 6793, 2761, 1144]}
ratio: 0.8262609297040817
Bleu-1 Score: 0.2761
Bleu-2 Score: 0.1852
Bleu-3 Score: 0.1259
Bleu-4 Score: 0.0868
METEOR Score: 0.1690
ROUGE_L Score: 0.2831
CIDEr Score: 0.8055
Accuracy: 0.4723


In [16]:
from nlgeval import NLGEval
import numpy as np

def get_nlg_scores_aggregated(references, hypotheses):
    """
    Tính toán điểm số NLG cho toàn bộ tập dữ liệu bằng cách tính tổng của tất cả các cặp câu trước, 
    sau đó chia cho số lượng cặp.

    Args:
    references (list): Danh sách các câu tham chiếu
    hypotheses (list): Danh sách các câu giả thuyết

    Returns:
    dict: Từ điển chứa các điểm số trung bình cho toàn bộ tập dữ liệu
    """
    nlgeval = NLGEval(no_skipthoughts=True, no_glove=True, metrics_to_omit=["SPICE"])
    
    # Tính toán điểm số cho từng cặp (tham chiếu, giả thuyết)
    all_scores = []
    for refs, hyp in zip(references, hypotheses):
        if isinstance(refs, str):
            refs = [refs]  # Đảm bảo refs luôn là một danh sách
        scores = nlgeval.compute_individual_metrics(refs, hyp)
        all_scores.append(scores)

    # Tính trung bình các điểm số
    avg_scores = {
        metric: np.mean([score[metric] for score in all_scores])
        for metric in all_scores[0]
    }

    # Chỉ giữ lại các metric phù hợp với VQA-X
    relevant_metrics = [
        "Bleu_1",
        "Bleu_2",
        "Bleu_3",
        "Bleu_4",
        "METEOR",
        "ROUGE_L",
        "CIDEr",
    ]

    return {k: v for k, v in avg_scores.items() if k in relevant_metrics}

# Chuẩn bị dữ liệu mẫu cho câu tham chiếu và dự đoán
target_explanations = merged_df['target_explanation'].tolist()
predicted_explanations = merged_df['explanation'].tolist()

# Tính toán các điểm số
scores = get_nlg_scores_aggregated(target_explanations, predicted_explanations)

# In ra các điểm số
print("Các điểm số trung bình cho toàn bộ tập dữ liệu:")
for metric, score in scores.items():
    print(f"{metric}: {score:.4f}")


Các điểm số trung bình cho toàn bộ tập dữ liệu:
Bleu_1: 0.2489
Bleu_2: 0.1472
Bleu_3: 0.0769
Bleu_4: 0.0397
METEOR: 0.1764
ROUGE_L: 0.2831
CIDEr: 0.0000


In [17]:
def compute_metrics_from_lists(hypotheses, references_list, no_overlap=False, no_skipthoughts=False, no_glove=False):
    """
    Hàm tính toán điểm số NLG trực tiếp từ danh sách các câu tham chiếu và giả thuyết.
    
    Args:
    hypotheses (list): Danh sách các câu giả thuyết
    references_list (list of list): Danh sách các danh sách câu tham chiếu
    """
    # Chuyển danh sách câu tham chiếu và giả thuyết thành dictionary
    refs = {idx: [line.strip() for line in strippedlines] for idx, strippedlines in enumerate(references_list)}
    hyps = {idx: [lines.strip()] for (idx, lines) in enumerate(hypotheses)}
    
    # print(hyps)
    # print(refs)
    # print(len(refs), len(hyps))

    assert len(refs) == len(hyps), "Số lượng câu tham chiếu và giả thuyết không khớp!"

    ret_scores = {}
    if not no_overlap:
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(), "METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr"),
        ]
        for scorer, method in scorers:
            score, scores = scorer.compute_score(refs, hyps)
            if isinstance(method, list):
                for sc, scs, m in zip(score, scores, method):
                    ret_scores[m] = sc
            else: 
                ret_scores[method] = score

        del scorers


    return ret_scores

# Chuẩn bị danh sách câu tham chiếu và giả thuyết
target_explanations = merged_df['target_explanation'].tolist()
predicted_explanations = merged_df['explanation'].tolist()

# Đảm bảo rằng target_explanations là danh sách của danh sách, mỗi câu giả thuyết có thể có nhiều tham chiếu
target_explanations_list = [[ref] for ref in target_explanations]

# Tính toán các điểm số
scores = compute_metrics_from_lists(predicted_explanations, target_explanations_list)

# In ra các điểm số
print("Các điểm số trung bình cho toàn bộ tập dữ liệu:")
for metric, score in scores.items():
    print(f"{metric}: {score:.4f}")

{'testlen': 50178, 'reflen': 60729, 'guess': [50178, 44305, 38432, 32559], 'correct': [17095, 6793, 2761, 1144]}
ratio: 0.8262609297040817
Các điểm số trung bình cho toàn bộ tập dữ liệu:
Bleu_1: 0.2761
Bleu_2: 0.1852
Bleu_3: 0.1259
Bleu_4: 0.0868
METEOR: 0.1690
ROUGE_L: 0.2831
CIDEr: 0.8055
